In [ ]:
# -*- coding: utf-8 -*-
# Find the time and value of max load for each of the regions
# COAST, EAST, FAR_WEST, NORTH, NORTH_C, SOUTHERN, SOUTH_C, WEST
# and write the result out in a csv file, using pipe character | as the delimiter.
# An example output can be seen in the "example.csv" file.

import xlrd
import os
import csv
from zipfile import ZipFile

datafile = "2013_ERCOT_Hourly_Load_Data.xls"
outfile = "2013_Max_Loads.csv"


def open_zip(datafile):
    with ZipFile('{0}.zip'.format(datafile), 'r') as myzip:
        myzip.extractall()




def parse_file(datafile):
    # YOUR CODE HERE
    # Opens up the data file to work on
    workbook = xlrd.open_workbook(datafile)
    # we specifiy the worksheet
    sheet = workbook.sheet_by_index(0)
    # Initial empty dictionary to store the data init.
    data = {}
    # Iterate over in the specified worksheet
    for i in range(1, 9):
        # get the cell value in first row and ith column and store it into variable station
        station = sheet.cell_value(0, i)
        # slice the value from ith row and all column and store it into variable d
        d = sheet.col_values(i, start_rowx=1, end_rowx=None)
        # gets the maximum of d and store it into maxi
        maxi = max(d)   
        # get the date of maxi     
        #maxiDate = xlrd.xldate_as_tuple(sheet.cell_value(d.index(maxi) + 1, 0),0)[:-2]
        maxpos = d.index(maxi) + 1
        maxtime = sheet.cell_value(maxpos, 0)
        time = xlrd.xldate_as_tuple(maxtime, 0)
        # store the maxi and maxi date into the dictionary as follows.
        data[station] = {"maxi": maxi, "maxtime": time}     
        # Iterate over when maximum column reached
    # Return the data.
    print data
    return data
# writing the files
def save_file(data, filename):
    # opens file for writing
    with open(filename, "w") as q:
        # csv.writer method has parameter which you can define delimiter as this: delimiter = '|'
        w = csv.writer(q, delimiter = '|')
        # Writes the row of corresponding
        w.writerow(["Station", "Year", "Month", "Day", "Hour", "Max Load"])
        # iterate over data
        for x in data:
            # Save the date as in year month, day, hour format.
            year, month, day, hour, _, _= data[x]['maxtime']
            # Then write that format also.
            w.writerow([x, year, month, day, hour, data[x]['maxi']])
    # YOUR CODE HERE above
def test():
    open_zip(datafile)
    data = parse_file(datafile)
    save_file(data, outfile)

    number_of_rows = 0
    stations = []

    ans = {'FAR_WEST': {'Max Load': '2281.2722140000024',
                        'Year': '2013',
                        'Month': '6',
                        'Day': '26',
                        'Hour': '17'}}
    correct_stations = ['COAST', 'EAST', 'FAR_WEST', 'NORTH',
                        'NORTH_C', 'SOUTHERN', 'SOUTH_C', 'WEST']
    fields = ['Year', 'Month', 'Day', 'Hour', 'Max Load']

    with open(outfile) as of:
        csvfile = csv.DictReader(of, delimiter="|")
        for line in csvfile:
            station = line['Station']
            if station == 'FAR_WEST':
                for field in fields:
                    # Check if 'Max Load' is within .1 of answer
                    if field == 'Max Load':
                        max_answer = round(float(ans[station][field]), 1)
                        max_line = round(float(line[field]), 1)
                        assert max_answer == max_line

                    # Otherwise check for equality
                    else:
                        assert ans[station][field] == line[field]

            number_of_rows += 1
            stations.append(station)

        # Output should be 8 lines not including header
        assert number_of_rows == 8

        # Check Station Names
        assert set(stations) == set(correct_stations)

        
if __name__ == "__main__":
    test()
